In [1]:
import os
import numpy as np
import pickle
import pandas as pd
from scipy.stats import zscore
from brainbox.io.one import SessionLoader
from sklearn.preprocessing import StandardScaler
import gc
import concurrent.futures

from functions import merge_licks, resample_common_time, interpolate_nans, low_pass
from one.api import ONE
one = ONE(mode='remote')

In [2]:
prefix = '/home/ines/repositories/'
# prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [4]:
""" Load BWM data post-QC """

data_path = prefix + '/representation_learning_variability/paper-individuality/'
filename = '1_bwm_qc_07-10-2025'



data_path = prefix + 'representation_learning_variability/Video and wheel/Video QC/'
filename = 'final_lp_qc07-10-2025'


bwm_query = pickle.load(open(data_path+filename, "rb"))


/tmp/ipykernel_10759/431073403.py:13: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  bwm_query = pickle.load(open(data_path+filename, "rb"))


In [3]:
extra_sessions = ['2d7c0f7f-e805-404b-914a-23d83998e08e',
'7691eeb3-715b-4571-8fda-6bb57aab8253',
'a28746ff-a6e0-403d-a11d-893c2f9a44b4',
'f140a2ec-fd49-4814-994a-fe3476f14e66',
'bd8b204f-a42e-45c1-a8f0-71c6223a6657',
'e4fac833-d985-4bb2-a97a-c4d7ed8d06e1',
'13b70283-e8d3-4e69-ae3f-83f50c9602e2',
'e34ee0ad-3ad8-4faa-b4d5-c1cc0cf3b496',
'f3eeb2d4-87ce-49ae-8a74-21665f6f1536',
'd9f0c293-df4c-410a-846d-842e47c6b502',
'86f741b4-9dee-4c28-8ee9-49f3656ab419',
'650a0a90-4bf3-4489-9bcd-75baf0a49eac',
'a2be3311-de26-48b3-bfc7-317e94a4fdd3',
'12f95449-6914-4d76-9eb3-7d79757a4a77',
'd9907ac3-7378-471d-91fb-c8ec34870e17',
'b4a8ec28-46ad-4b6f-908b-650d0970a0db',
'c51f34d8-42f6-4c9c-bb5b-669fd9c42cd9',
'71963e7f-5947-4c3e-bcc8-dfdef3d71be7',
'7471a326-894d-4b55-8b01-a0ad56af209c',
'8839a67d-991a-4694-9c45-b3af73ce4d36',
'db4fe6df-b1d2-4958-9c93-e71696d58f7f',
'80653a5b-c7aa-479d-9ae0-c92f296fface',
'169c9a39-cb63-4b77-93e2-10e076d4c472',
'495bee7e-b58e-42ea-8481-4a1bfedca54a',
'1db57661-5ad3-4465-b9ee-08473af9c2e8',
'3513e7f2-d2e6-4411-8055-54dac50458f6',
'd035c5ba-d51e-49a9-a94b-23531a598ec3',
'6a369bfa-a70b-4147-af25-d03772ff8d96',
'7050ae29-a99e-43f1-aa42-b4416200351c',
'3fa080ff-bcce-43e8-bd5f-601f0591f785',
'369c3073-e886-4b28-a32b-a5860df21392',
'aa8c915b-cc12-4022-8339-3faa438d7fe2',
'8b422ab2-fc98-4d25-ac1f-e239ca869d9e',
'87d86bb0-72d3-4213-be72-392295d3d601',
'2ab80a04-96f6-45f8-bf69-7eac67f81742',
'57701dff-8107-4ab6-ac7a-b087a2e4cc94',
'42541dce-2ffb-452e-bf54-119aee2ae48a',
'89848cb4-77d3-4db2-877e-2fa73a3cf5f6',
'33cbe984-deb7-489c-add6-d98ac9f64df6',
'0b5f5111-5647-4400-8e08-f57975027b5e',
'9e77877d-6fcf-4e91-9337-4b19277561d5',
'dc36f1b9-5dba-49c4-b333-ad08af6b8f86',
'8a039e2b-637e-45ed-8da6-0641924626f0',
'00d3c9ea-2c91-44c2-b03e-6dfec5e08f27',
'e698b903-98ed-48b9-bc40-664274f722da',
'e5094e8a-cd54-43d4-8a7a-c5aa54b15e19',
'09137957-7216-40ea-90b5-ef85a62b578a',
'03f0ed7f-e647-4732-ae1a-d41fc459138b',
'4373de88-6b08-4185-a224-f898fd0017d4',
'30c4e2ab-dffc-499d-aae4-e51d6b3218c2',
'1e07df0b-205b-4c94-8e02-2ae07dae4347',
'5c7d2345-1f0e-40e5-aad7-2c6133b71b09',
'1735d2be-b388-411a-896a-60b01eaa1cfe',
'14736609-bfdd-4620-8e47-09d7f4bc4412',
'4537bd9a-0a63-4462-870c-5f6d70d289ed',
'1715d2bc-4da3-4de2-9631-3ab1c600f2bf',
'aed404ce-b3fb-454b-ac43-2f12198c9eaf',
'f115196e-8dfe-4d2a-8af3-8206d93c1729',
'5c454bcb-ae77-42da-a8d2-b6463ea9f21b',
'c728f6fd-58e2-448d-aefb-a72c637b604c',
'c8fc81e8-fd09-4228-a08c-81c01f21381a',
'7502ae93-7437-4bcd-9e14-d73b51193656',
'297bd519-78f8-45d2-af85-835e865e228f',
'7b074b1a-6576-4380-91e4-ad6cdf06c3a6',
'064a7252-8e10-4ad6-b3fd-7a88a2db5463']

sessions_to_exclude = ['2d7c0f7f-e805-404b-914a-23d83998e08e', # bad right cam
'7691eeb3-715b-4571-8fda-6bb57aab8253', # bad view of paws
'bd8b204f-a42e-45c1-a8f0-71c6223a6657', # bad right camera
'f3eeb2d4-87ce-49ae-8a74-21665f6f1536', # moving licks
'650a0a90-4bf3-4489-9bcd-75baf0a49eac', # licks fail
'a2be3311-de26-48b3-bfc7-317e94a4fdd3', # few licks, few trials, ... but ok video; not enough behavior! 
'b4a8ec28-46ad-4b6f-908b-650d0970a0db', # few trials but ok!
'71963e7f-5947-4c3e-bcc8-dfdef3d71be7', # few trials but ok?
'495bee7e-b58e-42ea-8481-4a1bfedca54a', # timestamps
'1db57661-5ad3-4465-b9ee-08473af9c2e8', # timestamps
'6a369bfa-a70b-4147-af25-d03772ff8d96', # timestamps
'7050ae29-a99e-43f1-aa42-b4416200351c', # timestamps
'3fa080ff-bcce-43e8-bd5f-601f0591f785', # timestamps
'aa8c915b-cc12-4022-8339-3faa438d7fe2', # few trials but ok?
'8b422ab2-fc98-4d25-ac1f-e239ca869d9e', # few trials but ok?
'8a039e2b-637e-45ed-8da6-0641924626f0', # very few trials but ok?
'00d3c9ea-2c91-44c2-b03e-6dfec5e08f27', # very few trials but ok
'e5094e8a-cd54-43d4-8a7a-c5aa54b15e19', # a bit off timing; could be real??
'4373de88-6b08-4185-a224-f898fd0017d4', # dip in licking after start; can keep?
'5c7d2345-1f0e-40e5-aad7-2c6133b71b09',  # good
'5c454bcb-ae77-42da-a8d2-b6463ea9f21b', # bad licks
'c728f6fd-58e2-448d-aefb-a72c637b604c' # no data can be loaded
]

sessions_to_exclude = ['2d7c0f7f-e805-404b-914a-23d83998e08e', # bad right cam
'7691eeb3-715b-4571-8fda-6bb57aab8253', # bad view of paws
'bd8b204f-a42e-45c1-a8f0-71c6223a6657', # bad right camera
'f3eeb2d4-87ce-49ae-8a74-21665f6f1536', # moving licks
'650a0a90-4bf3-4489-9bcd-75baf0a49eac', # licks fail
'495bee7e-b58e-42ea-8481-4a1bfedca54a', # timestamps
'1db57661-5ad3-4465-b9ee-08473af9c2e8', # timestamps
'6a369bfa-a70b-4147-af25-d03772ff8d96', # timestamps
'7050ae29-a99e-43f1-aa42-b4416200351c', # timestamps
'3fa080ff-bcce-43e8-bd5f-601f0591f785', # timestamps
'5c454bcb-ae77-42da-a8d2-b6463ea9f21b', # bad licks
'c728f6fd-58e2-448d-aefb-a72c637b604c' # no data can be loaded
]



In [4]:
# Loop through animals
# sessions = bwm_query['eid'].unique()
sessions = extra_sessions

data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/extra_bwm/'
os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)

    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))

len(sessions_to_process)

3

In [5]:
paw_states = True

In [76]:
def process_design_matrix(session):

    file_path = one.eid2path(session)
    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    """ LOAD VARIABLES """
    sl = SessionLoader(eid=session, one=one)
    sl.load_pose(views=['left', 'right'], tracker='lightningPose')
    sl.load_session_data(trials=True, wheel=True, motion_energy=True)

    # Check if all data is available
    if np.sum(sl.data_info['is_loaded']) >= 4:

        # Poses
        poses = sl.pose
        lc_t = np.asarray(poses['leftCamera']['times'])
        rc_t = np.asarray(poses['rightCamera']['times'])
        left_fr = int(1/np.nanmean(np.diff(lc_t)))
        right_fr = int(1/np.nanmean(np.diff(rc_t)))
        
        # Motion energy
        me = sl.motion_energy
        mel_t = lc_t
        mer_t = rc_t
        motion_energy_l = interpolate_nans(me['leftCamera']['whiskerMotionEnergy'], left_fr)
        motion_energy_r = low_pass(interpolate_nans(me['rightCamera']['whiskerMotionEnergy'], right_fr), 
                                          cutoff=30, sf=right_fr)
        # Licks
        features = ['tongue_end_l_x', 'tongue_end_l_y', 'tongue_end_r_x', 'tongue_end_r_y']
        lick_t, licks = merge_licks(poses, features, common_fs=150)
        # Paws      
        l_paw_x = interpolate_nans(poses['leftCamera']['paw_r_x'], left_fr)
        l_paw_y = interpolate_nans(poses['leftCamera']['paw_r_y'], left_fr)
        r_paw_x = low_pass(interpolate_nans(poses['rightCamera']['paw_r_x'], right_fr), 
                                          cutoff=30, sf=right_fr)
        r_paw_y = low_pass(interpolate_nans(poses['rightCamera']['paw_r_y'], right_fr), 
                                          cutoff=30, sf=right_fr)
        l_paw_t = lc_t
        r_paw_t = rc_t
        # Wheel
        wheel = sl.wheel
        wheel_t = np.asarray(wheel['times'], dtype=np.float64)
        wheel_vel = wheel['velocity'].astype(np.float32)
        # Paw states
        if paw_states:
            times_l = one.load_dataset(session, '_ibl_%sCamera.times.npy' % 'left')
            times_r = one.load_dataset(session, '_ibl_%sCamera.times.npy' % 'right')
            assert np.all(lc_t==times_l)
            assert np.all(rc_t==times_r)
            paws_states_leftCamera = one.load_dataset(session, '_ibl_%sCamera.pawstates.pqt' % 'left')
            paws_states_rightCamera = one.load_dataset(session, '_ibl_%sCamera.pawstates.pqt' % 'right')
            paws_states_leftCamera['times'] = times_l
            paws_states_rightCamera['times'] = times_r

        # Common resampling window
        onset = max(lc_t.min(), rc_t.min(), wheel_t.min(), lick_t.min())
        offset = min(lc_t.max(), rc_t.max(), wheel_t.max(), lick_t.max())
        fs = 60
        ref_t = np.arange(onset, offset, 1 / fs, dtype=np.float64)

        # Restrict to time window
        def restrict(t, x):
            mask = (t >= onset) & (t <= offset)
            return t[mask], x[mask]

        mel_t, motion_energy_l = restrict(mel_t, motion_energy_l)
        mer_t, motion_energy_r = restrict(mer_t, motion_energy_r)
        wheel_t, wheel_vel = restrict(wheel_t, wheel_vel)
        l_paw_t_x, l_paw_x = restrict(l_paw_t, l_paw_x)
        l_paw_t_y, l_paw_y = restrict(l_paw_t, l_paw_y)
        r_paw_t_x, r_paw_x = restrict(r_paw_t, r_paw_x)
        r_paw_t_y, r_paw_y = restrict(r_paw_t, r_paw_y)
        lick_t, licks = restrict(lick_t, licks)

        # Resample
        mel_down, rt = resample_common_time(ref_t, mel_t, motion_energy_l, kind='linear')
        mer_down, _ = resample_common_time(ref_t, mer_t, motion_energy_r, kind='linear')
        wh_down, _ = resample_common_time(ref_t, wheel_t, wheel_vel, kind='linear')
        lk_down, _ = resample_common_time(ref_t, lick_t, licks, kind='nearest')
        lpx_down, _ = resample_common_time(ref_t, l_paw_t_x, l_paw_x, kind='linear')
        lpy_down, _ = resample_common_time(ref_t, l_paw_t_y, l_paw_y, kind='linear')
        rpx_down, _ = resample_common_time(ref_t, r_paw_t_x, r_paw_x, kind='linear')
        rpy_down, _ = resample_common_time(ref_t, r_paw_t_y, r_paw_y, kind='linear')

        # Create design matrix
        design_matrix = pd.DataFrame({
            'Bin': rt,
            'Lick count': lk_down.astype(np.int8),
            'avg_wheel_vel': wh_down,
            'whisker_me': mel_down,  # zscore(mel_down, nan_policy='omit'),
            'avg_whisker_me': np.nanmean([mel_down, mer_down], axis=0),  # zscore(np.nanmean([mel_down, mer_down], axis=0), nan_policy='omit'),
            'l_paw_x': lpx_down,
            'l_paw_y': lpy_down,
            'r_paw_x': rpx_down,
            'r_paw_y': rpy_down,
        })

        if paw_states:
            # Append paw states
            paw_vars = ['paw_r_still', 'paw_r_move', 'paw_r_wheel_turn', 'paw_r_groom',
                'paw_r_still_ens_var', 'paw_r_move_ens_var', 'paw_r_wheel_turn_ens_var', 'paw_r_groom_ens_var',
                'paw_l_still', 'paw_l_move', 'paw_l_wheel_turn', 'paw_l_groom',
                'paw_l_still_ens_var', 'paw_l_move_ens_var', 'paw_l_wheel_turn_ens_var', 'paw_l_groom_ens_var']
            for s, state in enumerate(paw_vars):
                resampled, _ = resample_common_time(ref_t, paws_states_leftCamera['times'], paws_states_leftCamera[state], kind='linear')
                var_name = 'leftCam_'+state
                design_matrix[var_name] = resampled
                resampled, _ = resample_common_time(ref_t, paws_states_rightCamera['times'], paws_states_rightCamera[state], kind='linear')
                var_name = 'rightCam_'+state
                design_matrix[var_name] = resampled
    
        # """ LOAD TRIAL DATA """
        session_trials = sl.trials
        session_start = list(session_trials['goCueTrigger_times'])[0]
        design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]

        """ SAVE DATA """       
        # Save unnormalized design matrix
        filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
        design_matrix.to_parquet(filename, compression='gzip')  

        # Save trials
        filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
        session_trials.to_parquet(filename, compression='gzip')  
        
        del design_matrix, session_trials, sl
        gc.collect()

    else:
        print('Data missing for session '+session)  


def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

In [85]:
for s, session in enumerate(sessions_to_process):
    try:    
        file_path = one.eid2path(session)
        if prefix == '/home/ines/repositories/':
            mouse_name = file_path.parts[8]
        else:
            mouse_name = file_path.parts[7]

        """ LOAD VARIABLES """
        sl = SessionLoader(eid=session, one=one)
        sl.load_pose(views=['left', 'right'], tracker='lightningPose')
        sl.load_session_data(trials=True, wheel=True, motion_energy=True)

        # Check if all data is available
        if np.sum(sl.data_info['is_loaded']) >= 4:

            # Poses
            poses = sl.pose
            lc_t = np.asarray(poses['leftCamera']['times'])
            rc_t = np.asarray(poses['rightCamera']['times'])
            left_fr = int(1/np.nanmean(np.diff(lc_t)))
            right_fr = int(1/np.nanmean(np.diff(rc_t)))
            
            # Motion energy
            me = sl.motion_energy
            mel_t = lc_t
            mer_t = rc_t
            motion_energy_l = interpolate_nans(me['leftCamera']['whiskerMotionEnergy'], left_fr)
            motion_energy_r = low_pass(interpolate_nans(me['rightCamera']['whiskerMotionEnergy'], right_fr), 
                                            cutoff=30, sf=right_fr)
            # Licks
            features = ['tongue_end_l_x', 'tongue_end_l_y', 'tongue_end_r_x', 'tongue_end_r_y']
            lick_t, licks = merge_licks(poses, features, common_fs=150)
            # Paws      
            l_paw_x = interpolate_nans(poses['leftCamera']['paw_r_x'], left_fr)
            l_paw_y = interpolate_nans(poses['leftCamera']['paw_r_y'], left_fr)
            r_paw_x = low_pass(interpolate_nans(poses['rightCamera']['paw_r_x'], right_fr), 
                                            cutoff=30, sf=right_fr)
            r_paw_y = low_pass(interpolate_nans(poses['rightCamera']['paw_r_y'], right_fr), 
                                            cutoff=30, sf=right_fr)
            l_paw_t = lc_t
            r_paw_t = rc_t
            # Wheel
            wheel = sl.wheel
            wheel_t = np.asarray(wheel['times'], dtype=np.float64)
            wheel_vel = wheel['velocity'].astype(np.float32)
            # Paw states
            if paw_states:
                times_l = one.load_dataset(session, '_ibl_%sCamera.times.npy' % 'left')
                times_r = one.load_dataset(session, '_ibl_%sCamera.times.npy' % 'right')
                assert np.all(lc_t==times_l)
                assert np.all(rc_t==times_r)
                paws_states_leftCamera = one.load_dataset(session, '_ibl_%sCamera.pawstates.pqt' % 'left')
                paws_states_rightCamera = one.load_dataset(session, '_ibl_%sCamera.pawstates.pqt' % 'right')
                paws_states_leftCamera['times'] = times_l
                paws_states_rightCamera['times'] = times_r

            # Common resampling window
            onset = max(lc_t.min(), rc_t.min(), wheel_t.min(), lick_t.min())
            offset = min(lc_t.max(), rc_t.max(), wheel_t.max(), lick_t.max())
            fs = 60
            ref_t = np.arange(onset, offset, 1 / fs, dtype=np.float64)

            # Restrict to time window
            def restrict(t, x):
                mask = (t >= onset) & (t <= offset)
                return t[mask], x[mask]

            mel_t, motion_energy_l = restrict(mel_t, motion_energy_l)
            mer_t, motion_energy_r = restrict(mer_t, motion_energy_r)
            wheel_t, wheel_vel = restrict(wheel_t, wheel_vel)
            l_paw_t_x, l_paw_x = restrict(l_paw_t, l_paw_x)
            l_paw_t_y, l_paw_y = restrict(l_paw_t, l_paw_y)
            r_paw_t_x, r_paw_x = restrict(r_paw_t, r_paw_x)
            r_paw_t_y, r_paw_y = restrict(r_paw_t, r_paw_y)
            lick_t, licks = restrict(lick_t, licks)

            # Resample
            mel_down, rt = resample_common_time(ref_t, mel_t, motion_energy_l, kind='linear')
            mer_down, _ = resample_common_time(ref_t, mer_t, motion_energy_r, kind='linear')
            wh_down, _ = resample_common_time(ref_t, wheel_t, wheel_vel, kind='linear')
            lk_down, _ = resample_common_time(ref_t, lick_t, licks, kind='nearest')
            lpx_down, _ = resample_common_time(ref_t, l_paw_t_x, l_paw_x, kind='linear')
            lpy_down, _ = resample_common_time(ref_t, l_paw_t_y, l_paw_y, kind='linear')
            rpx_down, _ = resample_common_time(ref_t, r_paw_t_x, r_paw_x, kind='linear')
            rpy_down, _ = resample_common_time(ref_t, r_paw_t_y, r_paw_y, kind='linear')

            # Create design matrix
            design_matrix = pd.DataFrame({
                'Bin': rt,
                'Lick count': lk_down.astype(np.int8),
                'avg_wheel_vel': wh_down,
                'whisker_me': mel_down,  # zscore(mel_down, nan_policy='omit'),
                'avg_whisker_me': np.nanmean([mel_down, mer_down], axis=0),  # zscore(np.nanmean([mel_down, mer_down], axis=0), nan_policy='omit'),
                'l_paw_x': lpx_down,
                'l_paw_y': lpy_down,
                'r_paw_x': rpx_down,
                'r_paw_y': rpy_down,
            })

            if paw_states:
                # Append paw states
                paw_vars = ['paw_r_still', 'paw_r_move', 'paw_r_wheel_turn', 'paw_r_groom',
                    'paw_r_still_ens_var', 'paw_r_move_ens_var', 'paw_r_wheel_turn_ens_var', 'paw_r_groom_ens_var',
                    'paw_l_still', 'paw_l_move', 'paw_l_wheel_turn', 'paw_l_groom',
                    'paw_l_still_ens_var', 'paw_l_move_ens_var', 'paw_l_wheel_turn_ens_var', 'paw_l_groom_ens_var']
                for s, state in enumerate(paw_vars):
                    resampled, _ = resample_common_time(ref_t, paws_states_leftCamera['times'], paws_states_leftCamera[state], kind='linear')
                    var_name = 'leftCam_'+state
                    design_matrix[var_name] = resampled
                    resampled, _ = resample_common_time(ref_t, paws_states_rightCamera['times'], paws_states_rightCamera[state], kind='linear')
                    var_name = 'rightCam_'+state
                    design_matrix[var_name] = resampled
        
            # """ LOAD TRIAL DATA """
            session_trials = sl.trials
            session_start = list(session_trials['goCueTrigger_times'])[0]
            design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]

            """ SAVE DATA """       
            # Save unnormalized design matrix
            filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
            design_matrix.to_parquet(filename, compression='gzip')  

            # Save trials
            filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
            session_trials.to_parquet(filename, compression='gzip')  
            
            del design_matrix, session_trials, sl
            gc.collect()

        else:
            print('Data missing for session '+session)  
    except Exception as exc:
        print(exc)
        print(s, session)
        pass

2026-02-18 12:03:06 INFO     one.py:1288 Loading trials data
2026-02-18 12:03:06 INFO     one.py:1288 Loading wheel data
2026-02-18 12:03:07 INFO     one.py:1288 Loading motion_energy data
2026-02-18 12:03:07 INFO     one.py:1288 Loading pupil data
2026-02-18 12:03:19 INFO     one.py:1288 Loading trials data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2026-02-18 12:03:20 INFO     one.py:1288 Loading wheel data
2026-02-18 12:03:20 INFO     one.py:1288 Loading motion_energy data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2026-01-21"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2026-01-26"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2026-02-18 12:03:21 INFO     one.py:1288 Loading pupil data


...

(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/leftCamera.ROIMotionEnergy.npy: 100%|██████████| 2.03M/2.03M [00:01<00:00, 1.48MB/s]
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/rightCamera.ROIMotionEnergy.npy: 100%|██████████| 5.08M/5.08M [00:01<00:00, 2.85MB/s]
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/bodyCamera.ROIMotionEnergy.npy: 100%|██████████| 1.02M/1.02M [00:01<00:00, 610kB/s]
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/_ibl_bodyCamera.times.npy: 100%|██████████| 1.02M/1.02M [00:00<00:00, 1.47MB/s]

2026-02-18 13:58:46 INFO     one.py:1288 Loading pupil data



(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/_ibl_leftCamera.features.pqt: 100%|██████████| 2.37M/2.37M [00:02<00:00, 934kB/s] 

2026-02-18 13:58:50 WARNING  one.py:1292 Could not load pupil data.



(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/lightningaction/_ibl_leftCamera.pawstates.pqt: 100%|██████████| 45.3M/45.3M [00:04<00:00, 11.3MB/s]
(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_029/2020-09-09/001/alf/lightningaction/_ibl_rightCamera.pawstates.pqt: 100%|██████████| 110M/110M [00:10<00:00, 10.6MB/s] 


In [77]:
for s, session in enumerate(sessions_to_process):
    process_design_matrix(session)

2026-02-18 11:46:10 INFO     one.py:1288 Loading trials data
2026-02-18 11:46:10 INFO     one.py:1288 Loading wheel data
2026-02-18 11:46:11 INFO     one.py:1288 Loading motion_energy data
2026-02-18 11:46:11 INFO     one.py:1288 Loading pupil data
2026-02-18 11:46:22 INFO     one.py:1288 Loading trials data
2026-02-18 11:46:22 INFO     one.py:1288 Loading wheel data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2026-02-18 11:46:23 INFO     one.py:1288 Loading motion_energy data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2026-01-21"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2026-01-26"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2026-02-18 11:46:23 INFO     one.py:1288 Loading pupil data


/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "", "2026-01-26"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:436: ALFWarning: Multiple revisions: "2026-02-12", ""
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2026-02-18 11:46:24 WARNING  one.py:1292 Could not load pupil data.


(S3) /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-30/2020-10-19/001/alf/lightningaction/_ibl_leftCamera.pawstates.pqt:  61%|██████    | 31.9M/52.6M [00:03<00:01, 12.2MB/s]